Bibliotecas

In [1]:
import pandas as pd
import os

In [2]:
# Caminho do arquivo CSV
caminho_arquivo = r'C:\Users\User\arq_tcc\empresas_setores.csv'

# Carregar o arquivo CSV em um DataFrame do pandas
cnpj_empresas = pd.read_csv(caminho_arquivo, encoding = 'utf-8', sep = ',')

In [ ]:
cnpj_empresas

In [3]:
cnpj_empresas.rename(columns = {'Denominação Social':' DENOM_CIA', 'CNPJ': 'CNPJ_CIA'},inplace=True)

In [13]:
# Lista de caminhos dos arquivos CSV
relatorios = ['EBITDA', 'ativo_circulante', 'ativo_total', 'df_numero_acoes',
              'lucro_acao', 'lucro_bruto', 'lucro_liquido', 'passivo_circulante',
              'passivo_exig_longo_prazo', 'patrimonio_liquido', 'vendas_liquidas', 'estoque', 'custo_prod_vendidos']

def carregar_relatorio(nome_relatorio):
    caminho_base = 'C:/Users/User/arq_tcc/Relatórios separados/'
    caminho_relatorio = f'{caminho_base}{nome_relatorio}.csv'

    try:
        df_relatorio = pd.read_csv(caminho_relatorio, encoding='utf-8', sep=',')
        return df_relatorio
    except FileNotFoundError:
        print(f"Arquivo não encontrado: {caminho_relatorio}")
        return None
    except Exception as e:
        print(f"Ocorreu um erro ao processar {caminho_relatorio}: {str(e)}")
        return None

# Relatórios:
nome_relatorio = relatorios[0]
EBITDA = carregar_relatorio(nome_relatorio)
EBITDA = EBITDA[EBITDA['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[1]
ativo_circulante = carregar_relatorio(nome_relatorio)
ativo_circulante = ativo_circulante[ativo_circulante['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[2]
ativo_total = carregar_relatorio(nome_relatorio)
ativo_total = ativo_total[ativo_total['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[5]
lucro_bruto = carregar_relatorio(nome_relatorio)
lucro_bruto = lucro_bruto[lucro_bruto['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[6]
lucro_liquido = carregar_relatorio(nome_relatorio)
lucro_liquido = lucro_liquido[lucro_liquido['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[7]
passivo_circulante = carregar_relatorio(nome_relatorio)
passivo_circulante = passivo_circulante[passivo_circulante['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[8]
passivo_exig_longo = carregar_relatorio(nome_relatorio)
passivo_exig_longo = passivo_exig_longo[passivo_exig_longo['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[9]
patrimonio_liquido = carregar_relatorio(nome_relatorio)
patrimonio_liquido = patrimonio_liquido[patrimonio_liquido['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[10]
vendas_liquidas = carregar_relatorio(nome_relatorio)
vendas_liquidas = vendas_liquidas[vendas_liquidas['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[11]
estoques = carregar_relatorio(nome_relatorio)
estoques = estoques[estoques['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

nome_relatorio = relatorios[12]
custo_prod_vendidos = carregar_relatorio(nome_relatorio)
custo_prod_vendidos = custo_prod_vendidos[custo_prod_vendidos['CNPJ_CIA'].isin(cnpj_empresas['CNPJ_CIA'])]

In [14]:
#concatenando os dfs
dataframes = [EBITDA, ativo_circulante, ativo_total,
              lucro_bruto, lucro_liquido, passivo_circulante,
              passivo_exig_longo, patrimonio_liquido, vendas_liquidas]

empresas_totais = pd.concat(dataframes, ignore_index=True)

In [15]:
caminho_base = 'C:/Users/User/arq_tcc/arq_zip'
nome_arquivo = 'empresas_totais.csv'
caminho_arquivo = os.path.join(caminho_base, nome_arquivo)

# Verifica se o arquivo já existe antes de salvar
if not os.path.exists(caminho_arquivo):
    # Salva o DataFrame como um arquivo CSV
    empresas_totais.to_csv(caminho_arquivo, index=False)
else:
    print(f"O arquivo {nome_arquivo} já existe no diretório especificado.")


Indicadores Financeiros para o TCC

ROA (Return on Assets)= lucro_liquido/ativos_totais

In [19]:
# Usando melt para transformar colunas dos anos em linhas
lucro_liquido_melted = lucro_liquido.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='lucro_liquido')

# Usando melt para transformar colunas dos anos em linhas
ativo_total_melted = ativo_total.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='ativo_total')

# Realizando o merge baseado em 'CNPJ_CIA' e 'ano'
ROA = pd.merge(lucro_liquido_melted, ativo_total_melted, on=['CNPJ_CIA', 'ano'])

# Calculando a coluna ROA
ROA['ROA'] = (ROA['lucro_liquido'] / ROA['ativo_total']) * 100

ROA = ROA.drop('DENOM_CIA_y', axis=1)

ROA = ROA.rename(columns={'DENOM_CIA_x': 'DENOM_CIA'})

In [44]:
caminho = r'C:\Users\User\arq_tcc\Relatórios separados\ROA_corrigido.csv'
ROA.to_csv(caminho, index=False, encoding='utf-8')


liquidez_corrente = ativo_circulante/passivo_circulante

In [20]:
# Usando melt para transformar colunas dos anos em linhas para ativo_circulante
ativo_circulante_melted = ativo_circulante.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='ativo_circulante')

# Usando melt para transformar colunas dos anos em linhas para passivo_circulante
passivo_circulante_melted = passivo_circulante.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='passivo_circulante')

# Realizando o merge baseado em 'CNPJ_CIA' e 'ano'
liquidez_corrente_df = pd.merge(ativo_circulante_melted, passivo_circulante_melted, on=['CNPJ_CIA', 'ano'])

# Calculando a coluna liquidez_corrente
liquidez_corrente_df['liquidez_corrente'] = liquidez_corrente_df['ativo_circulante'] / liquidez_corrente_df['passivo_circulante']

# Removendo a coluna 'DENOM_CIA' duplicada, se necessário
# Se você tiver colunas 'DENOM_CIA_x' e 'DENOM_CIA_y' após o merge, você pode querer manter uma e descartar a outra
liquidez_corrente_df = liquidez_corrente_df.drop(['DENOM_CIA_y'], axis=1, errors='ignore')
liquidez_corrente_df = liquidez_corrente_df.rename(columns={'DENOM_CIA_x': 'DENOM_CIA'})


CTPT = exigivel_total/patrimonio_liquido

Exigível_total = passivo_circulante + exig_longo_prazo

In [30]:
# Realizando o melt para transformar colunas dos anos em linhas para cada DataFrame
passivo_exig_longo_melted = passivo_exig_longo.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='passivo_exig_longo_valor')
passivo_circulante_melted = passivo_circulante.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='passivo_circulante_valor')
patrimonio_liquido_melted = patrimonio_liquido.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='patrimonio_liquido_valor')

# Realizando merges sucessivos baseado em 'CNPJ_CIA' e 'ano'
df_merged = pd.merge(passivo_exig_longo_melted, passivo_circulante_melted, on=['CNPJ_CIA', 'ano'])
df_merged = pd.merge(df_merged, patrimonio_liquido_melted, on=['CNPJ_CIA', 'ano'])

# Calculando 'exigivel_total' e 'cap_terc_patrimo_liquido'
df_merged['exigivel_total'] = df_merged['passivo_exig_longo_valor'] + df_merged['passivo_circulante_valor']
df_merged['cap_terc_patrimo_liquido'] = df_merged['exigivel_total'] / df_merged['patrimonio_liquido_valor']
CTPT_df = df_merged

CTPT_df = CTPT_df.drop(['DENOM_CIA_x','DENOM_CIA_y'], axis=1)

margem_liquida = lucro_liquido/receita_liquida

In [34]:
# Transformando lucro_liquido para o formato longo
lucro_liquido_melted = lucro_liquido.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='lucro_liquido_valor')

# Transformando vendas_liquidas para o formato longo
vendas_liquidas_melted = vendas_liquidas.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='vendas_liquidas_valor')

# Realizando o merge baseado em 'CNPJ_CIA' e 'ano'
margem_liquida_df = pd.merge(lucro_liquido_melted, vendas_liquidas_melted, on=['CNPJ_CIA', 'ano'])

# Calculando a coluna margem_liquida
margem_liquida_df['margem_liquida'] = (margem_liquida_df['lucro_liquido_valor'] / margem_liquida_df['vendas_liquidas_valor'])

# Limpando o DataFrame resultante, se necessário
# Removendo ou renomeando 'DENOM_CIA_x' e 'DENOM_CIA_y', se foram criadas pelo merge
margem_liquida_df.drop(['DENOM_CIA_y'], axis=1, inplace=True, errors='ignore')
margem_liquida_df.rename(columns={'DENOM_CIA_x': 'DENOM_CIA'}, inplace=True, errors='ignore')

In [42]:
CTPT_df

,CNPJ_CIA,ano,passivo_exig_longo_valor,passivo_circulante_valor,DENOM_CIA,patrimonio_liquido_valor,exigivel_total,cap_terc_patrimo_liquido
0,12.091.809/0001-55,2012,NaN,NaN,3R PETROLEUM ÓLEO E GÁS S.A.,NaN,NaN,NaN
1,37.663.076/0001-07,2012,NaN,NaN,AES BRASIL ENERGIA S.A.,NaN,NaN,NaN
2,21.240.146/0001-84,2012,NaN,NaN,AGROGALAXY PARTICIPAÇÕES S.A.,NaN,NaN,NaN
3,42.771.949/0001-35,2012,NaN,NaN,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,NaN,NaN,NaN
4,42.771.949/0001-35,2012,NaN,NaN,CENTRO DE IMAGEM DIAGNOSTICOS S.A.,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3460,67.571.414/0001-41,2022,149346.0,399038.0,VIVER INCORPORADORA E CONSTRUTORA S.A.,-181613.0,548384.0,-3.019520
3461,50.926.955/0001-42,2022,238951.0,569971.0,VULCABRAS S.A.,1711834.0,808922.0,0.472547
3462,84.429.695/0001-11,2022,2623428.0,10262877.0,WEG S.A.,15248355.0,12886305.0,0.845095
3463,33.130.691/0001-05,2022,2577183.0,805409.0,WILSON SONS S.A.,2348512.0,3382592.0,1.440313


margem_ebitda = EBITDA/vendas_liquidas

In [37]:
# Transformando EBITDA para o formato longo
EBITDA_melted = EBITDA.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='EBITDA_valor')

# Transformando vendas_liquidas para o formato longo (se ainda não estiver no formato necessário)
vendas_liquidas_melted = vendas_liquidas.melt(id_vars=['DENOM_CIA', 'CNPJ_CIA'], var_name='ano', value_name='vendas_liquidas_valor')

# Realizando o merge baseado em 'CNPJ_CIA' e 'ano'
margem_ebitda_df = pd.merge(EBITDA_melted, vendas_liquidas_melted, on=['CNPJ_CIA', 'ano'])

# Calculando a coluna margem_ebitda
margem_ebitda_df['margem_ebitda'] = (margem_ebitda_df['EBITDA_valor'] / margem_ebitda_df['vendas_liquidas_valor'])

# Limpando o DataFrame resultante, se necessário
# Removendo ou renomeando 'DENOM_CIA_x' e 'DENOM_CIA_y', se foram criadas pelo merge
margem_ebitda_df.drop('DENOM_CIA_y', axis=1, inplace=True, errors='ignore')
margem_ebitda_df.rename(columns={'DENOM_CIA_x': 'DENOM_CIA'}, inplace=True, errors='ignore')


In [46]:
liquidez_corrente_df

,DENOM_CIA,CNPJ_CIA,ano,ativo_circulante,passivo_circulante,liquidez_corrente
0,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,2012,NaN,NaN,NaN
1,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,2012,NaN,NaN,NaN
2,AGROGALAXY PARTICIPAÇÕES S.A.,21.240.146/0001-84,2012,NaN,NaN,NaN
3,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2012,NaN,NaN,NaN
4,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2012,NaN,NaN,NaN
...,...,...,...,...,...,...
2734,VIVER INCORPORADORA E CONSTRUTORA S.A.,67.571.414/0001-41,2022,204232.0,399038.0,0.511811
2735,VULCABRAS S.A.,50.926.955/0001-42,2022,1593107.0,569971.0,2.795067
2736,WEG S.A.,84.429.695/0001-11,2022,19653210.0,10262877.0,1.914981
2737,WILSON SONS S.A.,33.130.691/0001-05,2022,872083.0,805409.0,1.082783


Concatenado os dfs

In [51]:
# Inicialmente, temos ROA já definido. Vamos começar com o merge de ROA e margem_ebitda_df
df_merged = pd.merge(ROA, margem_ebitda_df[['CNPJ_CIA', 'ano', 'margem_ebitda']], on=['CNPJ_CIA', 'ano'], how='outer')

# Adicionando margem_liquida_df
df_merged = pd.merge(df_merged, margem_liquida_df[['CNPJ_CIA', 'ano', 'margem_liquida']], on=['CNPJ_CIA', 'ano'], how='outer')

# Adicionando liquidez_corrente_df
df_merged = pd.merge(df_merged, liquidez_corrente_df[['CNPJ_CIA', 'ano', 'liquidez_corrente']], on=['CNPJ_CIA', 'ano'], how='outer')

# Adicionando CTPT_df (cap_terc_patrimo_liquido)
df_merged = pd.merge(df_merged, CTPT_df[['CNPJ_CIA', 'ano', 'cap_terc_patrimo_liquido']], on=['CNPJ_CIA', 'ano'], how='outer')


In [52]:
caminho_arquivo = r'C:\Users\User\arq_tcc\arq_zip\empresas_totais_indices.csv'
df_merged.to_csv(caminho_arquivo, index=False)